Question1 : Rabbits Problem

In [3]:
class RabbitState:
    def __init__(self, state):
        self.state = state

    def __eq__(self, other):
        return isinstance(other, RabbitState) and self.state == other.state

    def __hash__(self):
        return hash(tuple(self.state))

    def __str__(self):
        return ''.join(self.state)

    def goalTest(self):
        return self.state == ['L', 'L', 'L', '_', 'R', 'R', 'R']

    def moveGen(self):
        next_states = []
        idx = self.state.index('_')
        directions = [-1, -2, 1, 2]
        for d in directions:
            new_idx = idx + d
            if 0 <= new_idx < 7:
                if abs(d) == 1 or (abs(d) == 2 and self.state[idx + d // 2] != '_'):
                    new_state = self.state.copy()
                    new_state[idx], new_state[new_idx] = new_state[new_idx], new_state[idx]
                    next_states.append(RabbitState(new_state))
        return next_states


BFS AND DFS

In [ ]:
def reconstructPath(goal_node_pair, CLOSED):
    parent_map = {}
    for node, parent in CLOSED:
        parent_map[node] = parent
    
    path = []
    goal_node, parent = goal_node_pair
    path.append(goal_node)
    while parent is not None:
        path.append(parent)
        parent = parent_map[parent]
    
    return path


def removeSeen(children, OPEN, CLOSED):
    open_nodes = [node for node, parent in OPEN]
    closed_nodes = [node for node, parent in CLOSED]
    new_nodes = [c for c in children if c not in open_nodes and c not in closed_nodes]
    return new_nodes
    

def bfs(start):
    OPEN = [(start, None)]
    CLOSED = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair
        
        if N.goalTest():
            print("Goal is found")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            for node in path:
                print(node, " -> ",end="")
            return
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(node, N) for node in new_nodes]
            OPEN = OPEN + new_pairs
    return []

#DFS
def dfs(start):
    OPEN = [(start, None)]
    CLOSED = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair
        
        if N.goalTest():
            print("Goal is found")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            for node in path:
                print(node, " -> ",end="")
            return
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(node, N) for node in new_nodes]
            OPEN = new_pairs + OPEN #LIFO
    return []

In [11]:
start_state = RabbitState(['R', 'R', 'R', '_', 'L', 'L', 'L'])

print("BFS:")
bfs(start_state)

print("\nDFS:")
dfs(start_state)


BFS:
Goal is found
RRR_LLL  -> RR_RLLL  -> RRLR_LL  -> RRLRL_L  -> RRL_LRL  -> R_LRLRL  -> _RLRLRL  -> LR_RLRL  -> LRLR_RL  -> LRLRLR_  -> LRLRL_R  -> LRL_LRR  -> L_LRLRR  -> LL_RLRR  -> LLLR_RR  -> LLL_RRR  -> 
DFS:
Goal is found
RRR_LLL  -> RR_RLLL  -> RRLR_LL  -> RRL_RLL  -> RR_LRLL  -> R_RLRLL  -> RLR_RLL  -> RL_RRLL  -> _LRRRLL  -> L_RRRLL  -> LR_RRLL  -> LRRR_LL  -> LRRRL_L  -> LRR_LRL  -> LR_RLRL  -> _RLRLRL  -> R_LRLRL  -> RL_RLRL  -> RLR_LRL  -> R_RLLRL  -> RR_LLRL  -> RRLL_RL  -> RRLLLR_  -> RRLLL_R  -> RRLL_LR  -> RR_LLLR  -> R_RLLLR  -> RLR_LLR  -> RL_RLLR  -> R_LRLLR  -> _RLRLLR  -> LR_RLLR  -> LRR_LLR  -> LRRL_LR  -> LR_LRLR  -> L_RLRLR  -> _LRLRLR  -> RL_LRLR  -> RLL_RLR  -> RLLLR_R  -> RLLL_RR  -> RLL_LRR  -> R_LLLRR  -> _RLLLRR  -> LR_LLRR  -> L_RLLRR  -> LLR_LRR  -> LL_RLRR  -> LLLR_RR  -> LLL_RRR  -> 

Question2 : Bridge Problem

In [8]:
class BridgeState:
    def __init__(self, Amogh, Ameya, Grandmother, Grandfather, time,umbrella_side):
        self.positions = {
            "Amogh": Amogh,
            "Ameya": Ameya,
            "Grandmother": Grandmother,
            "Grandfather": Grandfather
        }
        self.time = time
        self.umbrella_side=umbrella_side

    def __eq__(self, other):
        return self.positions == other.positions and self.time == other.time

    def __hash__(self):
        return hash((tuple(sorted(self.positions.items())), self.time,self.umbrella_side))

    def __str__(self):
        return f"{self.positions}, Time: {self.time}"

    def goalTest(self):
        return all(side == "right" for side in self.positions.values()) and self.time <= 60

    def moveGen(self):
        children = []
        times = {
            "Amogh": 5,
            "Ameya": 10,
            "Grandmother": 20,
            "Grandfather": 25
        }

        candidates = [p for p, side in self.positions.items() if side == self.umbrella_side]

        if self.umbrella_side == "left":
            for i in range(len(candidates)):
                for j in range(i+1, len(candidates)):
                    p1, p2 = candidates[i], candidates[j]
                    new_positions = self.positions.copy()
                    new_positions[p1] = "right"
                    new_positions[p2] = "right"
                    new_time = self.time+ max(times[p1], times[p2])
                    if new_time <= 60:
                        child = BridgeState(**new_positions, time=new_time,umbrella_side="right")
                        children.append(child)
        else:
            for p in candidates:
                new_positions = self.positions.copy()
                new_positions[p] = "left"
                new_time = self.time + times[p]
                if new_time <= 60:
                    child = BridgeState(**new_positions, time=new_time,umbrella_side="left")
                    children.append(child)

        return children

In [9]:
start = BridgeState(
    Amogh="left",
    Ameya="left",
    Grandmother="left",
    Grandfather="left",
    time=0,
    umbrella_side="left"
)
print("Bridge Problem - BFS")
bfs(start)

print("\nBridge Problem - DFS")
dfs(start)


Bridge Problem - BFS
Goal is found
{'Amogh': 'left', 'Ameya': 'left', 'Grandmother': 'left', 'Grandfather': 'left'}, Time: 0  -> {'Amogh': 'right', 'Ameya': 'right', 'Grandmother': 'left', 'Grandfather': 'left'}, Time: 10  -> {'Amogh': 'left', 'Ameya': 'right', 'Grandmother': 'left', 'Grandfather': 'left'}, Time: 15  -> {'Amogh': 'left', 'Ameya': 'right', 'Grandmother': 'right', 'Grandfather': 'right'}, Time: 40  -> {'Amogh': 'left', 'Ameya': 'left', 'Grandmother': 'right', 'Grandfather': 'right'}, Time: 50  -> {'Amogh': 'right', 'Ameya': 'right', 'Grandmother': 'right', 'Grandfather': 'right'}, Time: 60  -> 
Bridge Problem - DFS
Goal is found
{'Amogh': 'left', 'Ameya': 'left', 'Grandmother': 'left', 'Grandfather': 'left'}, Time: 0  -> {'Amogh': 'right', 'Ameya': 'right', 'Grandmother': 'left', 'Grandfather': 'left'}, Time: 10  -> {'Amogh': 'left', 'Ameya': 'right', 'Grandmother': 'left', 'Grandfather': 'left'}, Time: 15  -> {'Amogh': 'left', 'Ameya': 'right', 'Grandmother': 'right', '